In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

from random import randint

In [ ]:
dataset = pd.read_excel('/content/drive/MyDrive/Dataset.xlsx')
#dataset.head()

In [ ]:
X = dataset.drop('klass', axis=1)
Y = dataset['klass']

In [ ]:
def KNN(test_size, n, m, seed): # Деление на части, обучение и тестирование без и с нормализацией

  X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = test_size, random_state = seed)    

  classifier = KNeighborsClassifier(n_neighbors=n, p=m)
  classifier.fit(X_train, y_train)
  y_pred = classifier.predict(X_test)

  scaler = StandardScaler() # Нормализация
  scaler.fit(X_train)
  X_train = scaler.transform(X_train)
  X_test = scaler.transform(X_test)

  classifier = KNeighborsClassifier(n_neighbors=n, p=m)
  classifier.fit(X_train, y_train)
  y_pred_norm = classifier.predict(X_test)
  return y_test, y_pred, y_pred_norm

In [ ]:
neighbors = [] # количество соседей
#percent_test = [] # процент выборки для теста
#for_test = [] # количество для теста
#for_train = [] # количество для обучения
norm = [] # 0 - без нормализации, 1 - с нормализацией 
tn = [] # истинно отрицательный
tp = [] # истинно положительный
fn = [] # ложно отрицательный
fp = [] # ложно положительный
fails = [] # ошибки за один проход
percent = []
seed = [] # семя выборки
metric = [] # количество метрик
all = 7653
test_size = 0.2  
test_size_num = 1531

In [ ]:
def Temp(m):
  for s in range(0, 100): # сколько раз попробовать для каждых параметров
    random = randint(1, 2147483647)
    for n in range(1, 31): # количество соседей
      y_test, y_pred, y_pred_norm = KNN(test_size, n, m, random)

      norm.append(0)
      matrix = confusion_matrix(y_test, y_pred)
      tn.append(matrix[0][0])
      fn.append(matrix[0][1])
      fp.append(matrix[1][0])
      tp.append(matrix[1][1])
      fails.append(matrix[0][1] + matrix[1][0])
      percent.append(1 - (matrix[0][1] + matrix[1][0])/test_size_num)

      norm.append(1)
      matrix = confusion_matrix(y_test, y_pred_norm)
      tn.append(matrix[0][0])
      fn.append(matrix[0][1])
      fp.append(matrix[1][0])
      tp.append(matrix[1][1])
      fails.append(matrix[0][1] + matrix[1][0])
      percent.append(1 - (matrix[0][1] + matrix[1][0])/test_size_num)

      for i in range(0,2):
        neighbors.append(n)
        seed.append(random)
        metric.append(m)

In [ ]:
Temp(1)

In [ ]:
Temp(2)

In [ ]:
Temp(3)

In [ ]:
df = pd.DataFrame({'K-neighbors': neighbors,
                   'IsNormalization': norm,
                   'Metric': metric,
                   'TrueNegatives': tn,
                   'TruePositives': tp,
                   'FalseNegatives': fn,
                   'FalsePositives': fp,
                   'Fails': fails,
                   'PercentTrue': percent,
                   'Seed': seed})
df.to_excel('./Analysis.xlsx', index=False)

In [ ]:
zn = []
for j in range(0, 6):
  zn.append([])
  for i in range(0, 30):
    zn[j].append(0)
for i in range(0, 18000):
  zn[(metric[i]-1)*2+norm[i]][neighbors[i]-1]+= percent[i]
for i in zn:
  for j in i:
    j/=100

In [ ]:
newdf = pd.DataFrame({'K': range(1, 31),
                   'm1': zn[0],
                   'm1N': zn[1],
                   'm2': zn[2],
                   'm2N': zn[3],
                   'm3': zn[4],
                   'm3N': zn[5]})
newdf.to_excel('./Graph.xlsx', index=False)